In [1]:
import logging
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Audio
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import  r2_score
from sklearn.svm import SVR
from xgboost import XGBRegressor

from sklearn.model_selection import train_test_split
from scipy import signal
import math
import IPython
plt.style.use('seaborn')
def progress(value, max=100):
    return IPython.display.HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))

D:\Anaconda\envs\ai\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


# preprocess & create DB

In [2]:
def cut_off(series,cutsecond):
    for c,i in enumerate(series["port"]):
        i=float(i)
        if np.abs(i)<=0.001:
            temp=i
        else:
            data=series[c:]
            t=data['time'].iloc[0]
            #print("begining cut off index:{}  value:{} cutduration:{} threshold:{} remain samples:{}".format(c,i,t,temp,len(data)))
            break 
    try:        
        remain=cutsecond-float(t)  
        end=float(data['time'].iloc[-1])
        d=data
    except:
        #print("****")
        remain=cutsecond
        end=float(series['time'].iloc[-1])
        d=series
    counter=-1
    for c,i in reversed(list(enumerate(d['time']))):
        if remain-(end-float(i))<=0:
            data=d[:counter]
            #print("ending cut off index:{}  value:{} cutduration:{} remain samples:{}".format(c,i,remain,len(data))) 
            break
        counter=counter-1
    return data     

def normalize(dataframe):
  dataframe["port"]=np.float64(dataframe["port"])/abs((np.float64(dataframe["port"]))).max()
  return dataframe

def simple(samplerate,x):
    temp=signal.resample_poly(x, samplerate,len(x))
    return temp  

In [5]:
#samplerates=[50,100,200,400,800,1200,2400,4800]
samplerates=[400]
label_name="THREE_P2" #'ONE' , 'TWO' , 'THREE_P1','THREE_P2' , 'THREE_RESHAPE'
typee="z"
cs=3

if label_name=="THREE_P1" or label_name=="THREE_P2" or label_name=="THREE_RESHAPE":
    name=label_name+"_"+typee
else:
    name=label_name
    
def r2(y_pred):
    c,c2,index=0,0,0
    errors={}
    validation={}
    for i in y_pred:
        x,y,z=i
        x2,y2,z2=y_test.to_numpy()[c2]
        validation[c]=[x,y,z,index,'predict']
        c+=1
        validation[c]=[x2,y2,z2,index,'real']
        c+=1
        c2+=1
        index+=1
        errors[c]=math.sqrt((1/3)*((x2-x)**2+(y2-y)**2+(z2-z)**2))

    validation=pd.DataFrame.from_dict(validation, orient='index',columns=['x','y','z','index','kind'])
    logging.info('X={}'.format(np.corrcoef(validation[validation['kind'] == 'predict']['x'],validation[validation['kind'] == 'real']['x'])[0][1]))
    logging.info('Y={}'.format(np.corrcoef(validation[validation['kind'] == 'predict']['y'],validation[validation['kind'] == 'real']['y'])[0][1]))
    logging.info("Z={}".format(np.corrcoef(validation[validation['kind'] == 'predict']['z'],validation[validation['kind'] == 'real']['z'])[0][1]))
    
def load_db(samplerate):
    data={}    
    Data=pd.read_csv("DB_{}_SIGNALS.csv".format(name),index_col=0,header=0,compression='gzip')
    l=Data.columns.values.tolist()
    Data=Data.rename(columns={l[0]:"type"})
    del l[0]
    progressbar = display(progress(0, len(l)), display_id=True)
    for c,sample in enumerate(l):
        temp=Data.iloc[:,int(sample)]
        temp=temp.loc[temp.notna()]
        s,t = [],[]
        for idx, ele in enumerate(temp.tolist()): 
            if idx % 2 != 0: 
                t.append(ele) 
            else:
                s.append(ele)        
        df1={'port':s,'time':t}
        df1=pd.DataFrame.from_dict(df1)

        df1=normalize(df1)
        df1=cut_off(df1,10)  
        df1.reset_index(drop=True,inplace=True)

        data[c]=simple(samplerate,df1['port'])

        progressbar.update(progress(c+1, len(l)))


    dataframe=pd.DataFrame.from_dict(data, orient='index')
    df2=pd.read_csv("DB_{}_LOCATONS.csv".format(label_name))    

    #create labels
    y = pd.DataFrame({'x':df2['X']+450,
                  'y': df2['Y']+200,
                  'z': df2['Z']+200})

    Audio('alarm.mp3',autoplay=True)
    X_train, X_test, y_train, y_test = train_test_split(dataframe, y,random_state=11, test_size=0.2,shuffle=True)
    return X_train, X_test, y_train, y_test,dataframe.shape

# Linear regression

In [6]:
logging.basicConfig(filename="linear_reg.log",
                            filemode='a',
                            format=' %(asctime)s  %(message)s',
                            
                            level=logging.DEBUG)

for samplerate in samplerates:
    print(" At {} samplerate".format(samplerate))
    param_list = {"hidden_layer_sizes": [(1,),(50,)], "activation": ["tanh", "relu"], "solver": ["lbfgs", "sgd", "adam"], "alpha": [0.00005,0.0005]}
    regr = LinearRegression()
    param_grid = {
     'n_jobs': [50, 100,25,150],

     }
    X_train, X_test, y_train, y_test,sh=load_db(samplerate)
    gs_svr = GridSearchCV(regr,param_grid )
    clf =MultiOutputRegressor(gs_svr)
    clf = clf.fit(X_train,y_train)
    print(clf.estimators_[0].best_params_["n_jobs"] )

    regr = LinearRegression(n_jobs=clf.estimators_[0].best_params_["n_jobs"]) 
    regr =MultiOutputRegressor(regr)
    regr = regr.fit(X_train,y_train)

    y_pred = regr.predict(X_test)
    
    r2(y_pred)
    # The coefficients
    #print('Coefficients: \n', regr.coef_)
    # The mean squared error
    print('Mean squared error: %.2f'
          % mean_squared_error(y_test, y_pred))
    # The coefficient of determination: 1 is perfect prediction
    print('Coefficient of determination: %.2f'
          % r2_score(y_test, y_pred))
    print(regr.score(X_test,y_test))
    print("error",mean_squared_error(y_test, y_pred))
    logging.info(f"\n CS_{cs} Linear regression: \n dataset shape: {sh} \n Mean squared error: {mean_squared_error(y_test, y_pred)} \n samples: {samplerate} \n --------------------------------- \n")
Audio('alarm.mp3',autoplay=True)    

 At 400 samplerate


50
Mean squared error: 205901.06
Coefficient of determination: -5.48
-5.47675738145945
error 205901.06214095434


# XGboost

In [6]:
logging.basicConfig(filename="xgboost.log",
                            filemode='a',
                            format=' %(asctime)s  %(message)s',
                            
                            level=logging.DEBUG)
for samplerate in samplerates:
    print(" At {} samplerate".format(samplerate))
    regr = XGBRegressor()
    X_train, X_test, y_train, y_test,sh=load_db(samplerate)
    """
    param_grid = {
        'n_estimators': [700, 1000,1500,2000],
        'colsample_bytree': [0.7, 0.8],
        'max_depth': [10,15,20],
        'reg_alpha': [1.1, 1.2, 1.3],
        'reg_lambda': [1.1, 1.2, 1.3],
        'subsample': [0.7, 0.8, 0.9]
    }

    gs_svr = GridSearchCV(regr,param_grid )
    clf =MultiOutputRegressor(gs_svr)
    clf = clf.fit(X_train,y_train)
    print(clf.estimators_[0].best_params_ )
    
    regr = XGBRegressor(max_depth=clf.estimators_[0].best_params_['max_depth'],n_estimators=clf.estimators_[0].best_params_['n_estimators']) 
    """
    regr = XGBRegressor()
    regr =MultiOutputRegressor(regr)
    regr = regr.fit(X_train,y_train)

    y_pred = regr.predict(X_test)
    r2(y_pred)
    # The coefficients
    #print('Coefficients: \n', regr.coef_)
    # The mean squared error
    print('Mean squared error: %.2f'
          % mean_squared_error(y_test, y_pred))
    # The coefficient of determination: 1 is perfect prediction
    print('Coefficient of determination: %.2f'
          % r2_score(y_test, y_pred))
    print(regr.score(X_test,y_test))

    logging.info(f"\n CS_{cs} XGBoost: \n dataset shape: {sh} \n Mean squared error: {mean_squared_error(y_test, y_pred)} \n samples: {samplerate} \n --------------------------------- \n")
Audio('alarm.mp3',autoplay=True)    

 At 400 samplerate


D:\Anaconda\envs\ai\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


Mean squared error: 14357.69
Coefficient of determination: 0.34
0.34376001653123084


D:\Anaconda\envs\ai\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


# SVR

In [6]:
logging.basicConfig(filename="SVR.log",
                            filemode='a',
                            format=' %(asctime)s  %(message)s',
                            
                            level=logging.DEBUG)

for samplerate in samplerates:
    print(" At {} samplerate".format(samplerate))
    X_train, X_test, y_train, y_test,sh=load_db(samplerate)

    regr = SVR(C=1000,gamma=0.01,kernel='rbf') 
    regr =MultiOutputRegressor(regr)
    regr = regr.fit(X_train,y_train)

    y_pred = regr.predict(X_test)
    r2(y_pred)
    # The coefficients
    #print('Coefficients: \n', regr.coef_)
    # The mean squared error
    print('Mean squared error: %.2f'
          % mean_squared_error(y_test, y_pred))
    # The coefficient of determination: 1 is perfect prediction
    print('Coefficient of determination: %.2f'
          % r2_score(y_test, y_pred))
    print(regr.score(X_test,y_test))

    logging.info(f"\n CS{cs}\n dataset shape: {sh} \n Mean squared error: {mean_squared_error(y_test, y_pred)} \n samples: {samplerate} \n --------------------------------- \n")
Audio('alarm.mp3',autoplay=True)    

 At 400 samplerate


Mean squared error: 10093.66
Coefficient of determination: 0.59
0.5854904541182652
